# IS211 - Zehn: Couchbase

## Import modules

In [ ]:
from couchbase.cluster import Cluster, ClusterOptions
from couchbase.cluster import QueryOptions
from couchbase.auth import PasswordAuthenticator

## Thực hiện tạo mới dữ liệu trên Couchbase bằng Python


In [ ]:
cluster = Cluster('couchbase://26.167.214.119',
                  ClusterOptions(
                      PasswordAuthenticator('master', '123456')
                  )
)

collection = cluster.bucket('zehn').scope('zehn-scope').collection('zehn-scope-collection')

document = {
    "key1": "value1",
    "key2": 2,
    "key3": [3, 4, 5]
}

result = collection.insert("new-document", document)

print(result.cas)

## Truy vấn dữ liệu

### Truy vấn dữ liệu trực tiếp bằng Python không dùng N1QL Query


In [ ]:
cluster = Cluster('couchbase://26.167.214.119',
                  ClusterOptions(
                      PasswordAuthenticator('master', '123456')
                  )
)

collection = cluster.bucket('travel-sample').scope('inventory').collection('airport')

result = collection.get('airport_1257')

print(result.content_as[str])

### Truy vấn dữ liệu bằng Python dùng N1QL Query

In [ ]:
cluster = Cluster('couchbase://26.167.214.119',
                  ClusterOptions(
                      PasswordAuthenticator('master', '123456')
                  )
)

result = cluster.query(
    """SELECT * 
     FROM `travel-sample`.inventory.airport 
     WHERE airport.city = 'Los Angeles' """
)

for row in result:
    print(row)

### Truy vấn dữ liệu phân tán bằng Python dùng N1QL Query
Liệt kê các sân bay (airports) và địa điểm du lịch (landmarks) trong cùng một thành phố, với các thành phố này thuộc nước Mỹ , sắp xếp theo địa điểm du lịch.

In [ ]:
cluster = Cluster('couchbase://26.46.70.3',
                  ClusterOptions(
                      PasswordAuthenticator('master', '123456')
                  )
)

result = cluster.query(
    """SELECT DISTINCT 
        MIN(aport.airportname) AS Airport__Name,
        MIN(lmark.name) AS Landmark_Name,
        MIN(aport.tz) AS Landmark_Time
    FROM 
        `travel-sample`.inventory.airport aport
        RIGHT JOIN `travel-sample`.inventory.landmark lmark
            ON aport.city = lmark.city
            AND aport.country = "United States"
    GROUP BY lmark.name
    ORDER BY lmark.name
    LIMIT 3;"""
)

for row in result:
    print(row)